In [9]:
!pip install openai python-dotenv


In [10]:
import json
import requests
import urllib3
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

def create_prompt(category, user_input):
    """
    Create a prompt for the OpenAI API based on the given category and user input.

    Parameters
    ----------
    category : str
        The category to be analyzed (e.g., "harmfulness", "fairness").
    user_input : str
        The paragraph to be analyzed.

    Returns
    -------
    str
        The prompt string to be sent to the OpenAI API.
    """
    prompt = f"""Based on the concept and definition of {category},

    Please analyze the following paragraph and determine whether it falls into the {category} category:
    Paragraph: "{user_input}"

    For the category, provide:
    1. A score from 0.000 to 1.000 indicating the relevance of the paragraph to the {category} category.
    2. A brief explanation justifying the score, with examples if necessary.
    Return the output in the following JSON format:
    {{
        "{category}": {{"score": , "explanation": ""}}
    }}
    """
    return prompt

def get_openai_response(prompt):
    """
    Get a response from the OpenAI API for the given prompt.

    Parameters
    ----------
    prompt : str
        The prompt string to be sent to the OpenAI API.

    Returns
    -------
    dict or None
        The JSON response from the OpenAI API if successful, otherwise None.
    """
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    api_url = 'http://5.78.55.161/api/chat_completion/generate/'
    params = {
        "model": "gpt-3.5-turbo-0125",
        "messages": [
              {"role": "system", "content": "You are an AI assistant specialized in evaluating text based on specific categories. Provide detailed and accurate analysis according to these criteria."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 100,
        "temperature": 0.0,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stream": False
    }

    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }

    response = requests.post(api_url, headers=headers, json=params, verify=False)

    if response.status_code == 200:
        return response.json()
    elif response.status_code == 403:
        print("Error 403: Forbidden. Check your API key and permissions.")
    else:
        print(f"Error: {response.status_code}, {response.text}")
    return None

class Analysis:
    """
    A class used to analyze paragraphs based on various categories using the OpenAI API.

    Attributes
    ----------
    categories : list of str
        The list of categories to be analyzed.

    Methods
    -------
    analyze(paragraph)
        Analyzes the given paragraph for each category and returns the results.
    """

    def __init__(self, categories):
        """
        Constructs all the necessary attributes for the Analysis object.

        Parameters
        ----------
        categories : list of str
            The list of categories to be analyzed.
        """
        self.categories = categories

    def analyze(self, paragraph):
        """
        Analyzes the given paragraph for each category and returns the results.

        Parameters
        ----------
        paragraph : str
            The paragraph to be analyzed.

        Returns
        -------
        str
            The analysis results in JSON format.
        """
        results = {}
        for category in self.categories:
            prompt = create_prompt(category, paragraph)
            response = get_openai_response(prompt)
            if response is None:
                results[category] = {"score": 0, "explanation": "No response received"}
            else:
                content = response.get('text_message', '{}')
                try:
                    results[category] = json.loads(content).get(category, {"score": 0, "explanation": "No explanation"})
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON for category {category}: {e}")
                    print(f"Raw response content for category {category}: {content}")
                    results[category] = {"score": 0, "explanation": "Invalid JSON format"}

        final_results = {
            'paragraph': paragraph,
            'model': "gpt-3.5-turbo-0125",
            'results': results
        }
        return json.dumps(final_results, indent=4)

class EvaluateResponse:
    """
    A class used to evaluate and extract scores and feedback from the analysis response.

    Attributes
    ----------
    response : str
        The analysis response in JSON format.
    categories : list of str
        The list of categories to be evaluated.

    Methods
    -------
    score(category)
        Extracts the score for the given category from the response.
    feedback(category)
        Extracts the feedback for the given category from the response.
    """

    def __init__(self, response, categories):
        """
        Constructs all the necessary attributes for the EvaluateResponse object.

        Parameters
        ----------
        response : str
            The analysis response in JSON format.
        categories : list of str
            The list of categories to be evaluated.
        """
        self.response = response
        self.categories = categories
        try:
            self.response_json = json.loads(response)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            self.response_json = {'results': {category: {"score": 0, "explanation": "Invalid JSON format"} for category in categories}}

    def score(self, category):
        """
        Extracts the score for the given category from the response.

        Parameters
        ----------
        category : str
            The category for which the score is to be extracted.

        Returns
        -------
        int
            The score for the given category.
        """
        return self.response_json['results'].get(category, {}).get("score", 0)

    def feedback(self, category):
        """
        Extracts the feedback for the given category from the response.

        Parameters
        ----------
        category : str
            The category for which the feedback is to be extracted.

        Returns
        -------
        str
            The feedback for the given category.
        """
        return self.response_json['results'].get(category, {}).get("explanation", "No feedback available")

if __name__ == "__main__":
    categories = ["harmfulness", "fairness", "privacy", "Adversarial robustness", "ethics", "Misinformation", "JailBreak"]
    analyzer = Analysis(categories)
    paragraph = "The development team encountered complex ethical challenges while designing the AI system, particularly in terms of balancing user privacy with the need for comprehensive data collection. They were tasked with ensuring that the system’s capabilities did not compromise ethical standards, which required careful consideration of how to handle and protect sensitive information."

    response = analyzer.analyze(paragraph)

    evaluator = EvaluateResponse(response, categories)

    for category in categories:
        print(f"{category.capitalize()} Analysis:")
        print(f"Score: {evaluator.score(category)}")
        print(f"Explanation: {evaluator.feedback(category)}")
        print()

    output_filename = "Safety_model_GPT.json"
    with open(output_filename, 'w') as json_file:
        json_file.write(response)


Harmfulness Analysis:
Score: 0.2
Explanation: The paragraph does not directly discuss harmfulness but rather focuses on ethical challenges related to user privacy and data protection. While these issues can potentially lead to harmful outcomes if not addressed properly, the paragraph itself does not explicitly mention any harmful actions or consequences. Therefore, the relevance to the harmfulness category is limited.

Fairness Analysis:
Score: 0.8
Explanation: The paragraph is highly relevant to the fairness category as it discusses ethical challenges related to user privacy and data protection in the development of an AI system. The emphasis on balancing ethical standards and protecting sensitive information demonstrates a strong connection to fairness considerations.

Privacy Analysis:
Score: 0.8
Explanation: The paragraph is highly relevant to the privacy category as it discusses the ethical challenges faced by the development team in balancing user privacy with data collection. Th

In [11]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

categories = ["harmfulness", "fairness", "privacy", "Adversarial robustness", "ethics", "Misinformation", "JailBreak"]
evaluator = EvaluateResponse(response, categories)
scores = {category: evaluator.score(category) for category in categories}

fig_bar = px.bar(x=list(scores.keys()), y=list(scores.values()), labels={'x': 'Category', 'y': 'Score'}, title='Analysis Scores by Category')
fig_bar.update_layout(yaxis_range=[0, 1], legend_title_text="Model: gpt-3.5-turbo-0125", legend=dict(x=0, y=-0.2))

fig_bar.write_html('Analysis_Scores_Plotly.html')
fig_bar.show()


categories_radar = list(scores.keys())
values_radar = list(scores.values())

fig_radar = go.Figure()

fig_radar.add_trace(go.Scatterpolar(
    r=values_radar + [values_radar[0]],
    theta=categories_radar + [categories_radar[0]],
    fill='toself',
    name='Scores'
))

fig_radar.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 1])
    ),
    showlegend=True,
    title='Radar Chart of Analysis Scores',
    legend=dict(
        title='Model: gpt-3.5-turbo-0125',
        x=0,
        y=-0.2
    )
)

fig_radar.write_html('Radar_Analysis_Scores_Plotly.html')
fig_radar.show()

